In [1]:

fopen = open("20221001.as-rel2.txt", encoding="utf-8")

relationships = {}

for line in fopen:
    if line[0] != "#":
        line_split = line.split("|")
        as1 = line_split[0].strip("\n")
        as2 = line_split[1].strip("\n")
        reltype = int(line_split[2].strip("\n"))
        if as1 not in relationships:
            # [customer array, provider array]
            relationships[as1] = [set(), set()]
        if as2 not in relationships:
            # [customer array, provider array]
            relationships[as2] = [set(), set()]
        # If p2c
        if reltype == -1:
            # as1 has customer
            relationships[as1][0].add(as2)
            # as2 has a provider
            relationships[as2][1].add(as1)


# Find all AS's with no children
customers = {}
as_list = set()
for as_name in relationships:
    as_data = relationships[as_name]
    if len(as_data[0]) == 0:
        customers[as_name] = as_data[0]
        as_list.add(as_name)

for as_name in as_list:
    parents = relationships[as_name][1]
    for parent in parents:
        if parent in customers:
            customers[parent].add(as_name)
        else:
            customers[parent] = {as_name}
        as_list.add(as_name)
        for c in customers[as_name]:
            customers[parent].add(c)
            as_list.add(as_name)
sorted_customers = sorted(customers.items(), key=lambda key: len(key[1]), reverse=True)[:15]
print(sorted_customers)

[('3356', {'989', '399367', '199503', '393947', '46108', '201923', '48659', '48134', '21576', '53966', '394000', '47309', '13172', '209453', '32999', '62989', '53873', '11086', '262413', '32861', '23256', '395782', '16934', '54321', '52839', '22764', '97', '204558', '62743', '14182', '19769', '395341', '46127', '36312', '55095', '14034', '32300', '29818', '19773', '34893', '393579', '30525', '54836', '396957', '13782', '4983', '397346', '54016', '54230', '55057', '35204', '7324', '23351', '2571', '394163', '46847', '33307', '271106', '54526', '18547', '10857', '199339', '22353', '397581', '11699', '25753', '41721', '26989', '47762', '31869', '10949', '13433', '32696', '26114', '40493', '8769', '11862', '202167', '15347', '23436', '56898', '400138', '29283', '393558', '394298', '393340', '270840', '62861', '35834', '11396', '62992', '53477', '14161', '31404', '36575', '396403', '398762', '40248', '199883', '40346', '53017', '204698', '393356', '63161', '395273', '59675', '32325', '20610